In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import time
from dask.distributed import Client
import core as core

import warnings
warnings.filterwarnings("ignore")

In [2]:
# location of era5 data on teachinghub
path="~/LEHRE/msc-intro-comp-met-ex-w2024/data/era5/"



In [3]:
years = [2000]
months = range(6, 6+1)
files = []

files = [f"era5-{year}-{month:02}.nc" for year in years for month in months]

files_path = [path+f for f in files]

print(f"Load data for {years=} {months=}")
files_path

Load data for years=[2000] months=range(6, 7)


['~/LEHRE/msc-intro-comp-met-ex-w2024/data/era5/era5-2000-06.nc']

In [5]:
ds=xr.open_mfdataset(files_path, engine="h5netcdf", chunks={"valid_time":1e5} )
ds_1 = ds.sel(latitude=slice(70,35)).sel(longitude=slice(350,359.9))
ds_1['longitude'] = ds_1['longitude'] - 360

In [6]:
ds

<xarray.Dataset> Size: 15GB
Dimensions:     (valid_time: 720, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 6kB 2000-06-01 ... 2000-06-30T23:...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      (valid_time) <U4 12kB dask.array<chunksize=(720,), meta=np.ndarray>
Data variables:
    t2m         (valid_time, latitude, longitude) float32 3GB dask.array<chunksize=(720, 121, 240), meta=np.ndarray>
    ssrd        (valid_time, latitude, longitude) float32 3GB dask.array<chunksize=(720, 121, 240), meta=np.ndarray>
    ssrdc       (valid_time, latitude, longitude) float32 3GB dask.array<chunksize=(720, 121, 240), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 3GB dask.array<chunksize=(720, 121, 240), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 3GB dask.array<chunksize=(720, 121, 240), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-22T21:21 GRIB to CDM+CF via cfgrib-0.9.1...

In [11]:
ds_1 = ds.sel(latitude=slice(70,35)).sel(longitude=slice(350,359.9))
ds_1['longitude'] = ds_1['longitude'] - 360

ds_2 = ds.sel(latitude=slice(70,35)).sel(longitude=slice(0,30))

ds_eu = xr.concat([ds_1, ds_2], dim="longitude")

In [12]:
ds_eu

<xarray.Dataset> Size: 327MB
Dimensions:     (valid_time: 720, latitude: 141, longitude: 161)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 6kB 2000-06-01 ... 2000-06-30T23:...
  * latitude    (latitude) float64 1kB 70.0 69.75 69.5 69.25 ... 35.5 35.25 35.0
  * longitude   (longitude) float64 1kB -10.0 -9.75 -9.5 ... 29.5 29.75 30.0
    expver      (valid_time) <U4 12kB dask.array<chunksize=(720,), meta=np.ndarray>
Data variables:
    t2m         (valid_time, latitude, longitude) float32 65MB dask.array<chunksize=(720, 41, 40), meta=np.ndarray>
    ssrd        (valid_time, latitude, longitude) float32 65MB dask.array<chunksize=(720, 41, 40), meta=np.ndarray>
    ssrdc       (valid_time, latitude, longitude) float32 65MB dask.array<chunksize=(720, 41, 40), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 65MB dask.array<chunksize=(720, 41, 40), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 65MB dask.array<chunksize=(720, 41, 40), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-22T21:21 GRIB to CDM+CF via cfgrib-0.9.1...

In [13]:
ds_eu.to_netcdf('europe_subset.nc')